In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Setup environment
!apt-get -qq install xxd
!pip install pandas numpy matplotlib
!pip install tensorflow==2.0.0-rc1
!pip install tensorflow-gpu==2.0.0
!pip install sklearn

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

filename = "tar_10.csv"  #target file

df = pd.read_csv("/content/drive/MyDrive/revise/" + filename)
length = df.shape[1]
print(length)
df2 = df.iloc[5:35,0]  #set input dimension
for x in range(length-1) :
  df2=pd.concat([df2,df.iloc[5:35,x+1]])

print(df2)

df2.to_csv("tar.csv")

filename = "non_10.csv"  #target file

df = pd.read_csv("/content/drive/MyDrive/revise/" + filename)
length= df.shape[1]
print(length)
df2 = df.iloc[5:35,0]   #set input dimension
for x in range(length-1) :
  df2=pd.concat([df2,df.iloc[5:35,x+1]])

print(df2)

df2.to_csv("non.csv")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

print(f"TensorFlow version = {tf.__version__}\n")

# Set a fixed random seed value, for reproducibility, this will allow us to get
# the same random numbers each time the notebook is run
SEED = 1337
np.random.seed(SEED)
tf.random.set_seed(SEED)

# the list of pattern that data is available for
PATTERNS = [
    "tar",
    "non",
]

SAMPLES_PER_PATTERN = 30

NUM_PATTERNS = 2

# create a one-hot encoded matrix that is used in the output
ONE_HOT_ENCODED_PATTERNS = np.eye(NUM_PATTERNS)
inputs = []
outputs = []

# read each csv file and push an input and output
for pattern_index in range(NUM_PATTERNS):
  pattern = PATTERNS[pattern_index]
  print(f"Processing index {pattern_index} for pattern '{pattern}'.")
  
  output = ONE_HOT_ENCODED_PATTERNS[pattern_index]
  
  df = pd.read_csv(pattern + ".csv")
  
  # calculate the number of pattern recordings in the file
  num_recordings = int(df.shape[0] / SAMPLES_PER_PATTERN)
  
  print(f"\tThere are {num_recordings} recordings of the {pattern} pattern.")
  
  for i in range(num_recordings):
    tensor = []
    for j in range(SAMPLES_PER_PATTERN):
      index = i * SAMPLES_PER_PATTERN + j
      # normalize the input data, between 0 to 1:
      # - acceleration is between: -4 to +4
      # - gyroscope is between: -2000 to +2000
      tensor += [
         (df['0'][index])/200
      ]

    inputs.append(tensor)
    outputs.append(output)

# convert the list to numpy array
inputs = np.array(inputs)
outputs = np.array(outputs)

print("Data set parsing and preparation complete.")

In [ ]:
num_inputs = len(inputs)
randomize = np.arange(num_inputs)
np.random.shuffle(randomize)

 Swap the consecutive indexes (0, 1, 2, etc) with the randomized indexes
inputs = inputs[randomize]
outputs = outputs[randomize]

# Split the recordings (group of samples) into three sets: training, testing and validation
TRAIN_SPLIT = int(0.6 * num_inputs)
TEST_SPLIT = int(0.2 * num_inputs )
print(TRAIN_SPLIT)
print(TEST_SPLIT) 

inputs_train, inputs_test, inputs_validate = np.split(inputs, [TRAIN_SPLIT, TEST_SPLIT])
outputs_train, outputs_test, outputs_validate = np.split(outputs, [TRAIN_SPLIT, TEST_SPLIT])

print("Data set randomization and splitting complete.")

"""
# 5-Cross validaion
#1
TEST_1 = 0
TEST_2 = 30

inputs_train_a, inputs_test_a, inputs_train2_a = np.split(inputs, [TEST_1,TEST_2])
outputs_train_a, outputs_test_a, outputs_train2_a = np.split(outputs, [TEST_1,TEST_2])

inputs_train_a = np.vstack((inputs_train_a, inputs_train2_a))
outputs_train_a = np.vstack((outputs_train_a, outputs_train2_a))

#2
TEST_1 = 30
TEST_2 = 60

inputs_train_b, inputs_test_b, inputs_train2_b = np.split(inputs, [TEST_1,TEST_2])
outputs_train_b, outputs_test_b, outputs_train2_b = np.split(outputs, [TEST_1,TEST_2])

inputs_train_b = np.vstack((inputs_train_b, inputs_train2_b))
outputs_train_b = np.vstack((outputs_train_b, outputs_train2_b))

#3
TEST_1 = 60
TEST_2 = 90

inputs_train_c, inputs_test_c, inputs_train2_c = np.split(inputs, [TEST_1,TEST_2])
outputs_train_c, outputs_test_c, outputs_train2_c = np.split(outputs, [TEST_1,TEST_2])

inputs_train_c = np.vstack((inputs_train_c, inputs_train2_c))
outputs_train_c = np.vstack((outputs_train_c, outputs_train2_c))

#4
TEST_1 = 90
TEST_2 = 119

inputs_train_d, inputs_test_d, inputs_train2_d = np.split(inputs, [TEST_1,TEST_2])
outputs_train_d, outputs_test_d, outputs_train2_d = np.split(outputs, [TEST_1,TEST_2])

inputs_train_d = np.vstack((inputs_train_d, inputs_train2_d))
outputs_train_d = np.vstack((outputs_train_d, outputs_train2_d))

#5
TEST_1 = 119
TEST_2 = 148

inputs_train_e, inputs_test_e, inputs_train2_e = np.split(inputs, [TEST_1,TEST_2])
outputs_train_e, outputs_test_e, outputs_train2_e = np.split(outputs, [TEST_1,TEST_2])

inputs_train_e = np.vstack((inputs_train_e, inputs_train2_e))
outputs_train_e = np.vstack((outputs_train_e, outputs_train2_e))

print("Data set randomization and splitting complete.")
"""

Data set randomization and splitting complete.


In [ ]:
num_Train = 0.6 * num_inputs
num_Test = 0.2 * num_inputs
num_Validate = 0.2 * num_inputs

inputs_train_LSTM = np.reshape(inputs_train, (num_Train, SAMPLES_PER_PATTERNS, 1))
inputs_validate_LSTM = np.reshape(inputs_validate, (num_Validate, SAMPLES_PER_PATTERNS, 1))
inputs_test_LSTM = np.reshape(inputs_test, (num_Test, SAMPLES_PER_PATTERNS, 1))

"""
inputs_train_LSTM_a = np.reshape(inputs_train_a, (num_Train, SAMPLES_PER_PATTERN, 1))
inputs_test_LSTM_a = np.reshape(inputs_test_a, (num_Test, SAMPLES_PER_PATTERN, 1))

inputs_train_LSTM_b = np.reshape(inputs_train_b, (num_Train, SAMPLES_PER_PATTERN, 1))
inputs_test_LSTM_b = np.reshape(inputs_test_b, (num_Test, SAMPLES_PER_PATTERN, 1))

inputs_train_LSTM_c = np.reshape(inputs_train_c, (num_Train, SAMPLES_PER_PATTERN, 1))
inputs_test_LSTM_c = np.reshape(inputs_test_c, (num_Test, SAMPLES_PER_PATTERN, 1))

inputs_train_LSTM_d = np.reshape(inputs_train_d, (num_Train, SAMPLES_PER_PATTERN, 1))
inputs_test_LSTM_d = np.reshape(inputs_test_d, (num_Test, SAMPLES_PER_PATTERN, 1))

inputs_train_LSTM_e = np.reshape(inputs_train_e, (num_Train, SAMPLES_PER_PATTERN, 1))
inputs_test_LSTM_e = np.reshape(inputs_test_e, (num_Test, SAMPLES_PER_PATTERN, 1))
"""

In [ ]:
print(outputs.shape)
print(inputs.shape)

(148, 2)
(148, 30)


In [ ]:
#DNN for ErrP

from tensorflow.keras.metrics import AUC

for i in range(10):
  #with tf.device('/device:GPU:0'):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dense(8, activation='relu'))
    model.add(tf.keras.layers.Dense(2, activation='sigmoid'))
    model.compile(optimizer='adam', loss='mse', metrics=[AUC(name='auc'),'accuracy'])
    csv_logger = tf.keras.callbacks.CSVLogger('/content/drive/MyDrive/revise/training.csv', append = True)
    history = model.fit(inputs_train, outputs_train, epochs=33, batch_size=10, validation_data=(inputs_validate, outputs_validate), callbacks = [csv_logger])

In [ ]:
#LSTM for ErrP

from tensorflow.keras.metrics import AUC

for i in range(10):
  #with tf.device('/device:GPU:0'):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(90, input_shape=(SAMPLES_PER_PATTERNS,1)))
    model.add(tf.keras.layers.Dense(18, activation='relu'))
    model.add(tf.keras.layers.Dense(2, activation='sigmoid'))
    model.compile(optimizer='adam', loss='mse', metrics=[AUC(name='auc'),'accuracy'])
    csv_logger = tf.keras.callbacks.CSVLogger('/content/drive/MyDrive/revise/training_LSTM.csv', append = True)
    history = model.fit(inputs_train_LSTM, outputs_train, epochs=150, batch_size=10, validation_data=(inputs_test_LSTM_b, outputs_test_b), callbacks = [csv_logger])

In [ ]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model to disk
open("PATTERNS_model.tflite", "wb").write(tflite_model)
  
import os
basic_model_size = os.path.getsize("PATTERNS_model.tflite")
print("Model is %d bytes" % basic_model_size)

In [ ]:
!echo "const unsigned char model[] = {" > /content/model.h
!cat gesture_model.tflite | xxd -i      >> /content/model.h
!echo "};"                              >> /content/model.h

import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
print("\nOpen the side panel (refresh if needed). Double click model.h to download the file.")